In [19]:
import os
import sys
import importlib
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GaussianNoise, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import pandas as pd
import tensorflow as tf



In [3]:
# tf.config.optimizer.set_jit(True)

notebooks_dir = os.path.abspath('')
proj_dir = os.path.dirname(notebooks_dir)
raw_data_dir = os.path.join(proj_dir, 'raw_data')
src_dir = os.path.join(proj_dir, 'src')

# Add src_dir to the system path to import helper file
sys.path.insert(1, src_dir)
#import cutpaste_helper
#importlib.reload(cutpaste_helper)

def makeWindows(image_dict,num_windows, width, height):
    '''
    Parameters:
    image_dict: dictionary containing arrays that represent images
    num_windows: the number of windows wanted
    width: the width of the window
    height: the height of the window
    '''
    #Grab windows from the full images from train images
    window_images = []
    for key in image_dict:
    #The 100 windows per image in a list
        for i in range(0, num_windows):
            #Get the width and height of an image
            dimensions = image_dict[key].shape
            image_w = dimensions[1]
            image_h = dimensions[0]
            #Random coordinates
            random_x_coord = random.randint(0, image_w - width)
            random_y_coord = random.randint(0, image_h-height)
            #Takes window out of a random part of the images at the random coordinates
            window = image_dict[key][random_y_coord:random_y_coord + window_y, random_x_coord:random_x_coord + window_x]
            window_images.append(window)
    window_images = np.array(window_images)
    return window_images

def makeWhiteSquares(window_list):
    #Create test window images dict with white square defects
    defect_window_list = np.copy(window_list)
#Fill defect train window images
    for image in defect_window_list:
        #Random sized white square (2D array full of ones)
        sq_random_x = random.randint(5,25)
        sq_random_y = random.randint(5,25)
        white_square = np.ones((sq_random_y,sq_random_x))
        #Random coordinates for the square
        random_x_coord = random.randint(0,window_x-sq_random_x)
        random_y_coord = random.randint(0,window_y-sq_random_y)
        #Replace the coordinates with white square
        image[random_y_coord:random_y_coord + sq_random_y, random_x_coord: random_x_coord + sq_random_x,0] = white_square
    return defect_window_list

def copyPaste(image):
    copied_image = np.copy(image)
    #Random sized copy-pasted area (2D array)
    sq_random_x = random.randint(5,25)
    sq_random_y = random.randint(5,25)
    #Random coordinates for the square
    random_x_coord = random.randint(0,image.shape[1]-sq_random_x)
    random_y_coord = random.randint(0,image.shape[0]-sq_random_y)
    #Copy the area
    copyArea = image[random_y_coord:random_y_coord + sq_random_y, random_x_coord: random_x_coord + sq_random_x,0]
    #Paste coordinates
    random_x_coord = random.randint(0,window_x-sq_random_x)
    random_y_coord = random.randint(0,window_y-sq_random_y)
    #Paste the area
    copied_image[random_y_coord:random_y_coord + sq_random_y, random_x_coord: random_x_coord + sq_random_x,0] = copyArea
    return copied_image

def createLabels(image_list, defect_status):
    #Create labels for the images
    labels_list = []
    for image in image_list:
        if defect_status:
            labels_list.append(1)
        else:
            labels_list.append(0)
    labels_list = np.array(labels_list)
    return labels_list
        
def shuffleTwoArrays(image_list, label_list):
    #Shuffles the image and label arrays in the same way
    randomize = np.arange(len(label_list))
    np.random.shuffle(randomize)
    image_list = image_list[randomize]
    label_list = label_list[randomize]
    return (image_list, label_list)

def generateImages(normal_image_list, num_normal, num_defects):
    #A generator that generates images
    while True:
        generated_images = []
        generated_labels = []
        for i in range(num_normal):
            random_index = random.randint(0,len(normal_image_list)-1)
            generated_images.append(normal_image_list[random_index])
            generated_labels.append(0)
        for i in range(num_defects):
            random_index = random.randint(0,len(normal_image_list)-1)
            generated_images.append(copyPaste(normal_image_list[random_index]))
            generated_labels.append(1)
        generated_images = np.array(generated_images)
        generated_labels = np.array(generated_labels)
        generated_images, generated_labels = shuffleTwoArrays(generated_images, generated_labels)
        n = generated_images.shape[0]
        yield generated_images, generated_labels.reshape(n, 1)

def valGeneratorImages(test_x, test_y, num_desired):
    while True:
        generated_images = []
        generated_labels = []
        for i in range(num_normal):
            random_index = random.randint(0,len(test_x)-1)
            generated_images.append(test_x[random_index])
            generated_labels.append(test_y[random_index])
        yield generated_images, generated_labels



In [4]:
# Load dictionaries with TEM images
f_name = os.path.join(raw_data_dir, 'test_full_arrays')
#Dict with test image arrays (each key has a value of an array of numbers between 0 and 1)
test_full_arrays = pickle.load(open(f_name, "rb"))
f_name = os.path.join(raw_data_dir, 'train_full_arrays')
#Dict with train image arrays (each key has a value of an array of numbers between 0 and 1)
train_full_arrays = pickle.load(open(f_name, "rb"))

window_x = 118
window_y = 84
num_windows = 100

#Make testing window images and labels
test_window_images = makeWindows(test_full_arrays, num_windows, window_x, window_y)
test_window_labels = createLabels(test_window_images, False)

#Make training window images and labels
train_window_images = makeWindows(train_full_arrays, num_windows ,window_x, window_y)
train_window_labels = createLabels(train_window_images, False)



In [25]:
train_gen = generateImages(train_window_images, 32, 32)
val_gen = generateImages(test_window_images, 512, 512)

In [26]:
val_x, val_y =next(val_gen)

In [29]:
train_window_images.shape

(11200, 84, 118, 1)

In [20]:
#Stuff
filters = 32
filter_size = 3
pool_size = 2
epoch_number = 50
my_models = []
my_val_accuracy = []
my_val_loss = []

# Build the model.
train_gen = generateImages(train_window_images, 32, 32)
val_gen = generateImages(test_window_images, 32, 32)
# print(f'4 Layers Results: ')
# model = Sequential([
#   Conv2D(filters, filter_size, input_shape = (84, 118, 1)), 
#   MaxPooling2D(pool_size = pool_size),
#   Conv2D(filters * 2, filter_size),
#   MaxPooling2D(pool_size = pool_size),
#   Conv2D(filters * 4, filter_size),
#   MaxPooling2D(pool_size = pool_size),
#   Conv2D(filters * 8, filter_size),
#   MaxPooling2D(pool_size = pool_size),
#   Flatten(),
#   Dense(2, activation = 'sigmoid')
# ])

model = Sequential()
model.add(Conv2D(8, (3, 3), input_shape=(84, 118, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(32))
#     model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Compile the model.
model.compile(loss='binary_crossentropy',
                  optimizer='nadam',
                  metrics=['accuracy'])

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 82, 116, 8)        80        
_________________________________________________________________
batch_normalization_1 (Batch (None, 82, 116, 8)        32        
_________________________________________________________________
activation (Activation)      (None, 82, 116, 8)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 41, 58, 8)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 39, 56, 16)        1168      
_________________________________________________________________
batch_normalization_2 (Batch (None, 39, 56, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 39, 56, 16)       

In [ ]:
# Train the model.
# history = model.fit(
#   train_gen,
#   validation_data = val_gen,
#   epochs = epoch_number,
#   steps_per_epoch=11200/64,
#   validation_steps = 11200/64
# )

# Fit model
history = model.fit(
    x = train_gen,
    epochs=200,
    steps_per_epoch=len(train_window_images)//64,
    validation_data=(val_x, val_y),
    validation_batch_size=int(len(val_x)*0.50))

my_models.append(model)
my_val_accuracy.append(history.history['val_accuracy'][-1])
my_val_loss.append(history.history['val_loss'][-1])


Epoch 1/200
175/175 [==============================] - 38s 214ms/step - loss: 0.7138 - accuracy: 0.5470 - val_loss: 0.7410 - val_accuracy: 0.5000
Epoch 2/200
175/175 [==============================] - 37s 212ms/step - loss: 0.3080 - accuracy: 0.8688 - val_loss: 0.9103 - val_accuracy: 0.5674
Epoch 3/200
175/175 [==============================] - 37s 212ms/step - loss: 0.0950 - accuracy: 0.9696 - val_loss: 0.3540 - val_accuracy: 0.8535
Epoch 4/200
175/175 [==============================] - 37s 212ms/step - loss: 0.0537 - accuracy: 0.9840 - val_loss: 0.1042 - val_accuracy: 0.9648
Epoch 5/200
175/175 [==============================] - 37s 212ms/step - loss: 0.0429 - accuracy: 0.9881 - val_loss: 0.0938 - val_accuracy: 0.9678
Epoch 6/200
175/175 [==============================] - 37s 213ms/step - loss: 0.0399 - accuracy: 0.9890 - val_loss: 0.0914 - val_accuracy: 0.9707
Epoch 7/200
175/175 [==============================] - 37s 213ms/step - loss: 0.0334 - accuracy: 0.9920 - val_loss: 0.1030 -

In [ ]:
model.save(os.path.join(notebooks_dir, 'cutpaste_200epochs.SavedModel'))

In [ ]:
history = model.fit(
    x = train_gen,
    epochs=800,
    steps_per_epoch=len(train_window_images)//64,
    validation_data=(val_x, val_y),
    validation_batch_size=int(len(val_x)*0.50))

model.save(os.path.join(notebooks_dir, 'cutpaste_1000epochs.SavedModel'))

In [ ]:

# Predict on the first 5 test images.
predictions = model.predict(test_window_images[:5])

# Print our model's predictions.
print("Predictions")
print(np.argmax(predictions, axis=1))

# Check our predictions against the ground truths.
print("Real Labels")
print(test_window_labels[:5])